In [36]:
import os
import base64
import requests
import json
import base64 
import tempfile
from urllib.parse import urlparse
from urllib.parse import unquote

import pandas as pd
import numpy as np
from getpass import getpass
from glob import glob
from tqdm import tqdm
%load_ext autoreload
%autoreload 2
import os
import random
import sqlite3 
import numpy as np
import json
import math
from tqdm.notebook import tqdm
from tqdm import tqdm
tqdm.pandas()

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import Advan_operator as ad_op  
from datetime import datetime


pd.set_option('display.max_columns', None)


import logging
# Create a logger
logger_name = 'all_logger'
logger = logging.getLogger(logger_name)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
target_state = {
    "Pennsylvania": {
        "FIPS": "42",
        "Code": "PA"
    },
    "New York": {
        "FIPS": "36",
        "Code": "NY"
    },
    "New Jersey": {
        "FIPS": "34",
        "Code": "NJ"
    },
    "Rhode Island": {
        "FIPS": "44",
        "Code": "RI"
    },
    "Maine": {
        "FIPS": "23",
        "Code": "ME"
    },
    "Connecticut": {
        "FIPS": "09",
        "Code": "CT"
    },
    "Vermont": {
        "FIPS": "50",
        "Code": "VT"
    },
    "New Hampshire": {
        "FIPS": "33",
        "Code": "NH"
    },
    "Delaware": {
        "FIPS": "10",
        "Code": "DE"
    },
    "Massachusetts": {
        "FIPS": "25",
        "Code": "MA"
    },
    "West Virginia": {
        "FIPS": "54",
        "Code": "WV"
    },
    "Maryland": {
        "FIPS": "24",
        "Code": "MD"
    },
    "Virginia": {
        "FIPS": "51",
        "Code": "VA"
    },
    "Ohio": {
        "FIPS": "39",
        "Code": "OH"
    }
}
 
# target_state_FIPS = [value['FIPS'] for key, value in target_state.items()]
target_state_code= [value['Code'] for key, value in target_state.items()]
# 
target_state_code

['PA',
 'NY',
 'NJ',
 'RI',
 'ME',
 'CT',
 'VT',
 'NH',
 'DE',
 'MA',
 'WV',
 'MD',
 'VA',
 'OH']

# Using Neighborhood Patterns (CBG)

In [38]:
data_dir = r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'
save_dir = r'D:\SafeGraph\Advan_2024_API\Monthly_Patterns_distance_by_Neighorhood'

os.makedirs(save_dir, exist_ok=True)

year = 2023
month = 6

month_dir = os.path.join(data_dir, str(year), f'{month:02}', '01')
print("month_dir:", month_dir)

month_dir: K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01


In [39]:
use_cols =  ['AREA', 
             "REGION",
            # "STOPS_BY_DAY", 
            "RAW_STOP_COUNTS", 
            # "RAW_DEVICE_COUNTS", 
            # "MEDIAN_DWELL", 
            # "STOPS_BY_EACH_HOUR",
            "DISTANCE_FROM_HOME",
            ]



all_files = glob(os.path.join(month_dir, "*.gz"))
print("Found file count:", len(all_files))
print("Top and bottom file:")
print(all_files[0])
print(all_files[-1])

Found file count: 11
Top and bottom file:
K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01\Neighborhood_Patterns_US-DATE_RANGE_START-2023-06-01_1.csv.gz
K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01\Neighborhood_Patterns_US-DATE_RANGE_START-2023-06-01_9.csv.gz


In [40]:
df_list = []
for idx, f in enumerate(all_files):
    basename = os.path.basename(f)    
    print(f"Processing: {idx+1} / {len(all_files)}, {basename}")
    df = pd.read_csv(f, usecols=use_cols)

    mask = df['REGION'].isin(target_state_code)
    df = df[mask] 
    
    # df['total_travel_distance'] = df['RAW_VISIT_COUNTS'] * df['DISTANCE_FROM_HOME']
    # aggre_df = df[['RAW_VISIT_COUNTS', 'RAW_VISITOR_COUNTS', 'total_travel_distance']].sum().to_frame().T
    # df_list.append(aggre_df)
    df_list.append(df)
    
    # break

df_all = pd.concat(df_list)
new_fname = os.path.join(save_dir, f'distance_{year}_{month}.csv')
print("Saving file at:", new_fname)
df_all.to_csv(new_fname, index=False)
print("Done")
df_all

Processing: 1 / 11, Neighborhood_Patterns_US-DATE_RANGE_START-2023-06-01_1.csv.gz
Processing: 2 / 11, Neighborhood_Patterns_US-DATE_RANGE_START-2023-06-01_10.csv.gz
Processing: 3 / 11, Neighborhood_Patterns_US-DATE_RANGE_START-2023-06-01_11.csv.gz
Processing: 4 / 11, Neighborhood_Patterns_US-DATE_RANGE_START-2023-06-01_2.csv.gz
Processing: 5 / 11, Neighborhood_Patterns_US-DATE_RANGE_START-2023-06-01_3.csv.gz
Processing: 6 / 11, Neighborhood_Patterns_US-DATE_RANGE_START-2023-06-01_4.csv.gz
Processing: 7 / 11, Neighborhood_Patterns_US-DATE_RANGE_START-2023-06-01_5.csv.gz
Processing: 8 / 11, Neighborhood_Patterns_US-DATE_RANGE_START-2023-06-01_6.csv.gz
Processing: 9 / 11, Neighborhood_Patterns_US-DATE_RANGE_START-2023-06-01_7.csv.gz
Processing: 10 / 11, Neighborhood_Patterns_US-DATE_RANGE_START-2023-06-01_8.csv.gz
Processing: 11 / 11, Neighborhood_Patterns_US-DATE_RANGE_START-2023-06-01_9.csv.gz
Saving file at: D:\SafeGraph\Advan_2024_API\Monthly_Patterns_distance_by_Neighorhood\distance_

,AREA,RAW_STOP_COUNTS,DISTANCE_FROM_HOME,REGION
0,330199751001,2615,28.0,NH
4,340258074001,32998,2100.0,NJ
7,516500103102,2905,469.0,VA
9,360470495003,5871,1216.0,NY
13,250214401004,20222,3842.0,MA
...,...,...,...,...
12440,360670046002,6125,32.0,NY
12452,420110117012,15560,16024.0,PA
12453,517209601004,3404,10744.0,VA
12455,361190059022,8895,31.0,NY


In [41]:
new_fname = os.path.join(save_dir, f'distance_{year}_{month}.csv')

df = pd.read_csv(new_fname)
df

,AREA,RAW_STOP_COUNTS,DISTANCE_FROM_HOME,REGION
0,330199751001,2615,28.0,NH
1,340258074001,32998,2100.0,NJ
2,516500103102,2905,469.0,VA
3,360470495003,5871,1216.0,NY
4,250214401004,20222,3842.0,MA
...,...,...,...,...
63080,360670046002,6125,32.0,NY
63081,420110117012,15560,16024.0,PA
63082,517209601004,3404,10744.0,VA
63083,361190059022,8895,31.0,NY


In [43]:
df = df.dropna(subset=['RAW_STOP_COUNTS', 'AREA', 'DISTANCE_FROM_HOME']).sort_values('DISTANCE_FROM_HOME')
df['DISTANCE_FROM_HOME'] = df['DISTANCE_FROM_HOME'].astype(int)
df['RAW_STOP_COUNTS'] = df['RAW_STOP_COUNTS'].astype(int)

df

,AREA,RAW_STOP_COUNTS,DISTANCE_FROM_HOME,REGION
43473,440070003002,11033,1,RI
40888,360894906001,12540,2,NY
23819,440070174002,12557,2,RI
10586,90091941002,9459,2,CT
28626,360471184003,17704,2,NY
...,...,...,...,...
55267,230299900000,225,1175893,ME
7964,360610002011,4333,1670627,NY
59846,391535047003,7990,1793701,OH
7662,250173661004,2148,2983967,MA


In [44]:
visit_lists = []

for idx, row in tqdm(df.iterrows()):
    visit_lists.append([row['DISTANCE_FROM_HOME']] * row['RAW_STOP_COUNTS'])
     


63084it [00:12, 5128.52it/s]


In [45]:
visit_list = np.hstack(visit_lists)
len(visit_list), df['RAW_STOP_COUNTS'].sum()

(717057104, 717057104)

In [46]:
median_value = np.median(visit_list)

print("25%:", np.percentile(visit_list, 25))
print("50%:", median_value)
print("75%:", np.percentile(visit_list, 75))
print("Max:", np.max(visit_list))


25%: 513.0
50%: 3360.0
75%: 7976.0
Max: 3889833


In [ ]:
# 2023-08
# 25%: 624.0
# 50%: 3719.0
# 75%: 8586.0
# Max: 11147814

# 2023-07
# 25%: 482.0
# 50%: 3553.0
# 75%: 8644.0
# Max: 3804372

# 2023-06
# 25%: 513.0
# 50%: 3360.0
# 75%: 7976.0
# Max: 3889833



In [ ]:
STOP

# Using Monthly Patterns (POI)

In [ ]:
data_dir = r'D:\SafeGraph\Advan_2024_API\Monthly_Patterns'
save_dir = r'D:\SafeGraph\Advan_2024_API\Monthly_Patterns_distance'


os.makedirs(save_dir, exist_ok=True)

year = 2023
month = 8

month_dir = os.path.join(data_dir, str(year), f'{month:02}', '01')
print("month_dir:", month_dir)

In [ ]:
use_cols =  ['DISTANCE_FROM_HOME', 
             "RAW_VISIT_COUNTS",
            "RAW_VISITOR_COUNTS", 
             'PLACEKEY',
             'LATITUDE', 'LONGITUDE',
             'REGION',
             'ISO_COUNTRY_CODE',
]

all_files = glob(os.path.join(month_dir, "*.gz"))
print("Found file count:", len(all_files))
print("Top and bottom file:")
print(all_files[0])
print(all_files[-1])

In [ ]:
df_list = []
for idx, f in enumerate(all_files):
    basename = os.path.basename(f)    
    print(f"Processing: {idx+1} / {len(all_files)}, {basename}")
    df = pd.read_csv(f, usecols=use_cols)

    df = df.query("ISO_COUNTRY_CODE == 'US'")
    mask = df['REGION'].isin(target_state_code)
    df = df[mask].drop(columns=['ISO_COUNTRY_CODE'])
    
    # df['total_travel_distance'] = df['RAW_VISIT_COUNTS'] * df['DISTANCE_FROM_HOME']
    # aggre_df = df[['RAW_VISIT_COUNTS', 'RAW_VISITOR_COUNTS', 'total_travel_distance']].sum().to_frame().T
    # df_list.append(aggre_df)
    df_list.append(df)
    
    # break

df_all = pd.concat(df_list)
new_fname = os.path.join(save_dir, f'distance_{year}_{month}.csv')
print("Saving file at:", new_fname)
df_all.to_csv(new_fname, index=False)
print("Done")
df_all

In [ ]:
# df = df_all

# df['total_travel_distance'] = df['RAW_VISIT_COUNTS'] * df['DISTANCE_FROM_HOME']
# aggre_df = df[['RAW_VISIT_COUNTS', 'RAW_VISITOR_COUNTS', 'total_travel_distance']].sum().to_frame().T
# aggre_df['total_travel_distance'] / aggre_df['RAW_VISIT_COUNTS']

In [ ]:
new_fname = os.path.join(save_dir, f'distance_{year}_{month}.csv')

df = pd.read_csv(new_fname)
df

In [ ]:


df = df.dropna(subset=['DISTANCE_FROM_HOME', 'RAW_VISIT_COUNTS']).sort_values('DISTANCE_FROM_HOME')
df['DISTANCE_FROM_HOME'] = df['DISTANCE_FROM_HOME'].astype(int)
df['RAW_VISIT_COUNTS'] = df['RAW_VISIT_COUNTS'].astype(int)

df

In [ ]:
visit_lists = []

for idx, row in tqdm(df.iterrows()):
    visit_lists.append([row['DISTANCE_FROM_HOME']] * row['RAW_VISIT_COUNTS'])
    pass

# len(distance_list), df['RAW_VISIT_COUNTS'].sum()
# list_of_lists
visit_list = np.hstack(visit_lists)
len(visit_list), df['RAW_VISIT_COUNTS'].sum()

In [ ]:
median_value = np.median(visit_list)

print("25%:", np.percentile(visit_list, 25))
print("Median:", median_value)
print("75%:", np.percentile(visit_list, 75))
print("Max:", np.max(visit_list))


In [ ]:
# 2023-06
# 25%: 7892.0
# Median: 12778.0
# 75%: 20498.0
# Max: 16179651

# 2023-07
# 25%: 8136.0
# Median: 13359.0
# 75%: 22957.0
# Max: 14179255

# 2023-08
# 25%: 8370.0
# Median: 13518.0
# 75%: 21588.0
# Max: 14404597

In [ ]:
arr = np.array(distance_list)
arr.shape

In [ ]:
df = pd.read_csv(r"D:\SafeGraph\Advan_2024_API\Monthly_Patterns\2023\06\01\Monthly_Patterns_Foot_Traffic-DATE_RANGE_START-2023-06-01_22.csv.gz")  # , usecols=use_cols
df

In [ ]:
df.columns

In [ ]:

df['total_travel_distance'] = df['RAW_VISIT_COUNTS'] * df['DISTANCE_FROM_HOME']
aggre_df = df[['RAW_VISIT_COUNTS', 'RAW_VISITOR_COUNTS', 'total_travel_distance']].sum().to_frame().T

In [ ]:
df